In [1]:
pip install boto3 langchain psycopg2-binary pandas faiss-cpu


  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/4e/a5/e6f90b0a768560a0c44cb9076a313ee6d669ec98fd2747a8451832403ffe/boto3-1.36.11-py3-none-any.whl.metadata
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/e9/65/e5cc2876078fa5f1a621c8429f0174855c7e9831060d350626dbf8d2a10c/langchain-0.3.17-py3-none-any.whl.metadata
  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/61/69/3b3d7bd583c6d3cbe5100802efa5beacaacc86e37b653fc708bf3d6853b8/psycopg2_binary-2.9.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/2c/2d/d2a4171a9cca9a7c04cd9d6f9441a37f1e0558724b90bf7fc7db08553601/faiss_cpu-1.10.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for botocore<1.37.0,>=1.36.11 from https://files.pythonhosted.org/packages/2c/ce/e97be00389d51a010c0680ea688a073737ca3b2de6f9

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.36.11 which is incompatible.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.10.6 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.12.0 which is incompatible.


In [4]:
import boto3
import os

# Option 1: Use default credential provider chain
session = boto3.Session()

print(session)

Session(region_name='us-west-2')


# Option 2: Use specific profile
session = boto3.Session(profile_name='default')

# Option 3: Use environment variables
os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_key'
os.environ['AWS_DEFAULT_REGION'] = 'your-region-1'

All available models in my region: 

In [6]:
import boto3

bedrock = boto3.client('bedrock', region_name='us-west-2')  # change to your region
models = bedrock.list_foundation_models()

for model in models['modelSummaries']:
    print(f"Model ID: {model['modelId']}")

Model ID: amazon.titan-tg1-large
Model ID: amazon.titan-embed-g1-text-02
Model ID: amazon.titan-text-lite-v1:0:4k
Model ID: amazon.titan-text-lite-v1
Model ID: amazon.titan-text-express-v1:0:8k
Model ID: amazon.titan-text-express-v1
Model ID: amazon.nova-pro-v1:0
Model ID: amazon.nova-lite-v1:0
Model ID: amazon.nova-micro-v1:0
Model ID: amazon.titan-embed-text-v1:2:8k
Model ID: amazon.titan-embed-text-v1
Model ID: amazon.titan-embed-text-v2:0
Model ID: amazon.titan-embed-image-v1:0
Model ID: amazon.titan-embed-image-v1
Model ID: amazon.titan-image-generator-v1:0
Model ID: amazon.titan-image-generator-v1
Model ID: amazon.titan-image-generator-v2:0
Model ID: amazon.rerank-v1:0
Model ID: stability.stable-diffusion-xl-v1:0
Model ID: stability.stable-diffusion-xl-v1
Model ID: stability.sd3-large-v1:0
Model ID: stability.sd3-5-large-v1:0
Model ID: stability.stable-image-core-v1:0
Model ID: stability.stable-image-core-v1:1
Model ID: stability.stable-image-ultra-v1:0
Model ID: stability.stable

In [ ]:
import boto3
import json

def simple_titan_agent(prompt: str) -> str:
    # Initialize Bedrock Runtime client in us-west-2
    bedrock = boto3.client(
        service_name='bedrock-runtime',
        region_name='us-west-2'
    )
    
    # Prepare the request for Titan
    request_body = {
        "inputText": prompt,
        "textGenerationConfig": {
            "maxTokenCount": 100,
            "temperature": 0,
            "topP": 1,
            "stopSequences": []
        }
    }
    
    # Use Titan Express model
    model_id = 'amazon.titan-text-express-v1'
    
    # Invoke the model
    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body)
    )
    
    # Parse and return the response
    response_body = json.loads(response.get('body').read())
    return response_body.get('results', [{}])[0].get('outputText', '')

# Test the agent
if __name__ == "__main__":
    test_prompt = "What are three interesting facts about the Pacific Northwest?"
    
    try:
        response = simple_titan_agent(test_prompt)
        print("\nPrompt:", test_prompt)
        print("\nTitan's response:", response)
    except Exception as e:
        print(f"An error occurred: {str(e)}")


Prompt: What are three interesting facts about the Pacific Northwest?

Titan's response: 
Here are three interesting facts about the Pacific Northwest: 

1. "The Pacific Northwest is one of the most beautiful places in the United States. Located along the coast of the Pacific Ocean, it includes Washington, Oregon, and Idaho. The region is known for its stunning scenery, including mountains, forests, and waterways. Here are three interesting facts about the Pacific Northwest:
2. The Cascade Range is one of the most prominent mountain ranges in the Pacific Northwest. It includes several volcanoes, including Mount St. Helens and Mount Rainier, and offers some of the best skiing and hiking in the United States.
3. The region is also home to several indigenous tribes, including the Nez Perce, Flathead, and Salish. These tribes have a rich history and culture and continue to play an important role in the Pacific Northwest today." 


In [19]:
import boto3

# Initialisation du client Redshift Data
client = boto3.client('redshift-data', region_name='us-west-2')

# Paramètres de connexion
database = 'dev'  # Remplace par le nom de ta base de données
workgroup_name = 'wz-solutions-redshift-workgroup'  # Remplace par ton workgroup

# Requête simple pour tester la connexion
sql_query = 'SELECT 1;'

try:
    response = client.execute_statement(
        Database=database,
        WorkgroupName=workgroup_name,  # Ne spécifie plus db_user ici
        Sql=sql_query
    )
    print("Connexion réussie !")
except Exception as e:
    print(f"Erreur de connexion : {e}")

{'CreatedAt': datetime.datetime(2025, 2, 1, 21, 38, 14, 565000, tzinfo=tzlocal()), 'Database': 'dev', 'DbUser': 'IAM:child-account-21-user-1', 'Id': '970d035b-afe0-49ce-af60-d965dad6b91a', 'WorkgroupName': 'wz-solutions-redshift-workgroup', 'ResponseMetadata': {'RequestId': '970d035b-afe0-49ce-af60-d965dad6b91a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '970d035b-afe0-49ce-af60-d965dad6b91a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '180', 'date': 'Sat, 01 Feb 2025 20:38:14 GMT'}, 'RetryAttempts': 0}}
Connexion réussie !


In [ ]:
import boto3
import time

# Initialisation du client Redshift Data
client = boto3.client('redshift-data', region_name='us-west-2')

# Paramètres de connexion
database = 'dev'  # Remplace par le nom de ta base de données
workgroup_name = 'wz-solutions-redshift-workgroup'  # Remplace par ton workgroup

# Requête SQL à exécuter
sql_query = 'SELECT * FROM consoma LIMIT 10;'  # Remplace par ta table

try:
    # Exécuter la requête SQL
    response = client.execute_statement(
        Database=database,
        WorkgroupName=workgroup_name,  # Ne spécifie plus db_user ici
        Sql=sql_query
    )
    
    # Récupérer l'ID de l'exécution de la requête
    statement_id = response['Id']
    print(f"Requête envoyée, ID : {statement_id}")

    # Attendre que la requête soit terminée
    while True:
        status_response = client.describe_statement(Id=statement_id)
        status = status_response['Status']

        if status in ['FINISHED', 'FAILED', 'ABORTED']:
            break
        print("En attente des résultats...")
        time.sleep(2)  # Pause avant la prochaine vérification

    # Vérifier si la requête s'est bien exécutée
    if status == 'FINISHED':
        # Récupérer les résultats
        result_response = client.get_statement_result(Id=statement_id)
        records = result_response.get('Records', [])

        # Afficher les résultats
        if records:
            print("\nRésultats de la requête :")
            for row in records:
                print([col.get('stringValue', 'NULL') for col in row])  # Adaptation pour afficher chaque ligne
        else:
            print("Aucun résultat trouvé.")

    else:
        print(f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}")

except Exception as e:
    print(f"Erreur de connexion ou d'exécution : {e}")


Requête envoyée, ID : 979c0d74-e9e9-427c-97ce-50786006a8a5
En attente des résultats...

Résultats de la requête :
['GN_1180180102051605', 'GN|1180180102051605|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180102342604', 'GN|1180180102342604|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180102501103', 'GN|1180180102501103|22910|1|00000', '2022-10-10', 'NULL', '2022-03-27', '2022-09-02', 'NULL', 'NULL', 'NULL']
['GN_1180180102596604', 'GN|1180180102596604|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180102978210', 'GN|1180180102978210|22910|1|00000', '2022-10-10', 'NULL', '2022-03-27', '2022-09-02', 'NULL', 'NULL', 'NULL']
['GN_1180180103044102', 'GN|1180180103044102|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180103080601', 'GN|1180180103080601|22910|1|00000', '2022-10-10', '

In [23]:
import boto3
import json
import time

# ---- AWS CONFIGURATION ----
AWS_REGION = "us-west-2"
MODEL_ID = "amazon.titan-text-express-v1"

# ---- REDSHIFT SERVERLESS CONFIGURATION ----
DATABASE = "dev"  # Remplace par ta base de données
WORKGROUP_NAME = "wz-solutions-redshift-workgroup"  # Remplace par ton workgroup

# ---- INITIALISATION DES CLIENTS ----
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)
redshift_client = boto3.client("redshift-data", region_name=AWS_REGION)


# ---- FONCTION POUR INTERROGER REDSHIFT ----
def query_redshift(sql_query):
    """Exécute une requête SQL sur Redshift Serverless et retourne les résultats."""
    try:
        response = redshift_client.execute_statement(
            Database=DATABASE,
            WorkgroupName=WORKGROUP_NAME,
            Sql=sql_query
        )

        statement_id = response['Id']
        print(f"Requête envoyée, ID : {statement_id}")

        while True:
            status_response = redshift_client.describe_statement(Id=statement_id)
            status = status_response["Status"]

            if status in ["FINISHED", "FAILED", "ABORTED"]:
                break
            print("En attente des résultats...")
            time.sleep(2)

        if status == "FINISHED":
            result_response = redshift_client.get_statement_result(Id=statement_id)
            records = result_response.get("Records", [])
            return records if records else "Aucun résultat trouvé."
        else:
            return f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}"

    except Exception as e:
        return f"Erreur de connexion ou d'exécution : {str(e)}"


# ---- FONCTION POUR ANALYSER LES DONNÉES AVEC AMAZON TITAN ----
def analyze_data_with_titan(prompt):
    """Envoie un prompt structuré à Amazon Titan pour détecter les anomalies et proposer des actions correctives."""
    request_body = {
        "inputText": prompt,
        "textGenerationConfig": {
            "maxTokenCount": 300,
            "temperature": 0,
            "topP": 1,
            "stopSequences": []
        }
    }

    response = bedrock.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(request_body)
    )

    response_body = json.loads(response["body"].read())
    return response_body.get("results", [{}])[0].get("outputText", "")


# ---- FONCTION PRINCIPALE : ANALYSE DE QUALITÉ DES DONNÉES ----
def data_quality_analysis(table_name):
    """Analyse la qualité des données d'une table Redshift et détecte des anomalies."""
    sql_query = f"SELECT * FROM {table_name} LIMIT 10;"
    records = query_redshift(sql_query)

    if isinstance(records, str):
        return records

    formatted_data = "\n".join([", ".join([col.get('stringValue', 'NULL') for col in row]) for row in records])

    prompt = f"""
    Tu es un expert en qualité des données, spécialisé dans l'analyse de tables Redshift.
    
    Voici un échantillon de la table "{table_name}":
    {formatted_data}
    
    1. Identifie les anomalies ou incohérences dans ces données.
    2. Génère une requête SQL pour approfondir l'analyse.
    3. Suggère des bonnes pratiques pour améliorer la qualité des données.

    Format attendu :
    - Anomalies détectées
    - Requête SQL de validation
    - Recommandations d'amélioration
    """

    return analyze_data_with_titan(prompt)


# ---- EXÉCUTION ----
if __name__ == "__main__":
    table_name = "abonnements"  # Remplace par le nom de ta table Redshift
    response = data_quality_analysis(table_name)

    print("\n📊 Rapport de Qualité des Données 📊\n")
    print(response)


Requête envoyée, ID : 7267a7c4-ae12-4479-9a26-6323221fe820
En attente des résultats...

📊 Rapport de Qualité des Données 📊


Anomalies détectées :
- Des dates de début de contrat manquantes.
- Des dates de fin de contrat manquantes.
- Des dates de début de contrat antérieures à la date de fin de contrat.

Requête SQL de validation :
SELECT *
FROM abonnements
WHERE date_de_fin_contrat < date_de_debut_contrat;

Recommandations d'amélioration :
- Vérifier et corriger les dates de début et de fin de contrat manquantes.
- Vérifier la cohérence des dates de début et de fin de contrat.
- Enrichir la table avec des informations complémentaires sur les contrats.


PROSPER: 

In [25]:
import boto3
import json
import time

# ---- AWS CONFIGURATION ----
AWS_REGION = "us-west-2"
MODEL_ID = "mistral.mixtral-8x7b-instruct-v0:1"  # Utilisation de Mistral AI

# ---- REDSHIFT SERVERLESS CONFIGURATION ----
DATABASE = "dev"  # Remplace par ta base de données
WORKGROUP_NAME = "wz-solutions-redshift-workgroup"  # Remplace par ton workgroup

# ---- INITIALISATION DES CLIENTS ----
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)
redshift_client = boto3.client("redshift-data", region_name=AWS_REGION)


# ---- FONCTION POUR RÉCUPÉRER LES TABLES REDSHIFT ----
def get_redshift_tables():
    """Récupère la liste des tables disponibles dans la base de données Redshift."""
    sql_query = "SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'public';"
    try:
        response = redshift_client.execute_statement(
            Database=DATABASE,
            WorkgroupName=WORKGROUP_NAME,
            Sql=sql_query
        )

        statement_id = response['Id']
        while True:
            status_response = redshift_client.describe_statement(Id=statement_id)
            status = status_response["Status"]
            if status in ["FINISHED", "FAILED", "ABORTED"]:
                break
            time.sleep(2)

        if status == "FINISHED":
            result_response = redshift_client.get_statement_result(Id=statement_id)
            tables = [row[0].get('stringValue', 'NULL') for row in result_response.get("Records", [])]
            return tables if tables else "Aucune table trouvée."
        else:
            return f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}"

    except Exception as e:
        return f"Erreur de connexion ou d'exécution : {str(e)}"


# ---- FONCTION POUR RÉCUPÉRER LES COLONNES D'UNE TABLE ----
def get_table_columns(table_name):
    """Récupère les colonnes et leur description d'une table Redshift."""
    sql_query = f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{table_name}';
    """
    try:
        response = redshift_client.execute_statement(
            Database=DATABASE,
            WorkgroupName=WORKGROUP_NAME,
            Sql=sql_query
        )

        statement_id = response['Id']
        while True:
            status_response = redshift_client.describe_statement(Id=statement_id)
            status = status_response["Status"]
            if status in ["FINISHED", "FAILED", "ABORTED"]:
                break
            time.sleep(2)

        if status == "FINISHED":
            result_response = redshift_client.get_statement_result(Id=statement_id)
            columns = [
                f"{row[0].get('stringValue', 'NULL')} ({row[1].get('stringValue', 'NULL')})"
                for row in result_response.get("Records", [])
            ]
            return columns if columns else "Aucune colonne trouvée."
        else:
            return f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}"

    except Exception as e:
        return f"Erreur de connexion ou d'exécution : {str(e)}"


# ---- FONCTION POUR ANALYSER LES TABLES AVEC L'AGENT ----
def agent_analyze_tables():
    """Analyse et décrit chaque table détectée dans Redshift."""
    tables = get_redshift_tables()
    if isinstance(tables, str):
        return tables

    all_tables_info = []
    for table in tables:
        columns = get_table_columns(table)
        all_tables_info.append(f"Table: {table}\nColonnes: {', '.join(columns)}")

    # Construire le prompt pour Mistral AI
    prompt = f"""
    Tu es un expert en bases de données. Analyse et décris chacune des tables trouvées dans Amazon Redshift.
    
    Voici les informations trouvées :
    {chr(10).join(all_tables_info)}

    Pour chaque table :
    1. Donne son rôle dans la base de données.
    2. Explique à quoi sert chaque colonne.
    
    Réponds de manière détaillée et pédagogique.
    """

    return analyze_with_mistral(prompt)


# ---- FONCTION POUR INTERAGIR AVEC MISTRAL AI ----
def analyze_with_mistral(prompt):
    """Envoie un prompt à Mistral AI via Amazon Bedrock."""
    request_body = {
        "prompt": prompt,
        "max_tokens": 1000,
        "temperature": 0.3, 
        "top_p": 0.9
    }

    response = bedrock.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(request_body)
    )

    response_body = json.loads(response["body"].read())
    return response_body.get("outputs", [{}])[0].get("text", "")


# ---- EXÉCUTION ----
if __name__ == "__main__":
    response = agent_analyze_tables()
    print("\n📊 Analyse des Tables Redshift 📊\n")
    print(response)



📊 Analyse des Tables Redshift 📊


    Table: abonnements

    1. Le rôle de cette table dans la base de données est de stocker les informations relatives aux abonnements des clients. Cela inclut des détails tels que le numéro d'abonnement, les dates d'entrée, de résiliation et de souscription.

    2. Les colonnes de cette table sont :

    - cle_abonnement (character varying) : Cette colonne stocke le numéro d'abonnement unique pour chaque client.
    
    - date_entree_local_abonnement (date) : Cette colonne enregistre la date à laquelle l'abonnement a été souscrit par le client.
    
    - date_resiliation_abonnement (date) : Cette colonne stocke la date à laquelle l'abonnement a été résilié par le client. Si l'abonnement est toujours actif, cette valeur sera NULL.
    
    - date_souscription_abonnement (date) : Cette colonne enregistre la date à laquelle l'abonnement a été souscrit par le client. Cette valeur devrait être identique à celle stockée dans la colonne date_entree_loca